<a href="https://colab.research.google.com/github/wesley34/comp3414_course_material/blob/master/ch11_python_Tensorflow_2.0/em_routing_capsnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [2]:
from tensorflow.keras import layers,models,Model,Input,regularizers


In [3]:
# scope 1
def scope_1(shape):
  input = Input(batch_size=batch_size,shape=shape,name="First_Block_Input")
  output = layers.Conv2D(A,[5,5],strides=2,padding="VALID",activation="relu",name="First_block_CNN_1")(input)  # None,28,28,5
  data_size = int(np.floor(shape[0]-5+1)/2) # None,12,12,32

  model = Model(inputs=input,outputs=output,name="Scope_1")
  data_shape = [data_size,data_size,A]
  return model,data_shape

In [4]:
# scope 2
def scope_2 (shape_0,shape_1,shape_2): # usually shape_0 == shape_1 , shape_2 is output channel
  shape = [shape_0,shape_1,shape_2]
  input = Input(batch_size=batch_size,shape=shape,name="Second_Block_Input")

  pose = layers.Conv2D(B*16,[1,1],name="Second_Block_CNN_1")(input) #None,28,28,8*16
  pose = tf.reshape(pose,[batch_size,shape_0,shape_1,B,16],"Second_Block_Reshape_1") #None,28,28,8,16

  activated = layers.Conv2D(B,[1,1],activation="sigmoid",name="Second_Block_CNN_2")(input) #None,28,28,8
  activated = tf.reshape(activated,[batch_size,shape_0,shape_1,B,1])  #None,28,28,8,1
  
  #concated
  output = tf.concat([pose,activated],axis=4,name="Second_Block_CNN_Plus_Activation") #None,28,28,8,17
  output = tf.reshape(output,[batch_size,shape_0,shape_1,-1],"Second_Block_Reshape_1") #None,28,28,8*17

  model = Model(inputs=input,outputs=output,name="Scope_2")
  data_size = [shape_0,shape_1,8*17]
  return model,data_size



In [5]:
# scope conv_caps 1


In [6]:
def build_kernel_tile(input_shape,kernel_size,stride,name):
 
  input = Input(shape=input_shape,batch_size=batch_size,name="Input_"+name) #None,28,28,8*17
  input_channel = input_shape[2] # input_shape = None,28,28,8*17
  # input ch is input_shape[2], output ch is kernel*kernel
  tile_filter = np.zeros([kernel_size,kernel_size,input_shape[2],kernel_size*kernel_size],dtype=np.float32) #3,3,,8*17,3*3

  for i in range(kernel_size):
    for j in range(kernel_size):
      tile_filter[i,j,:,i*kernel_size+j]=1.0 # set different [1] in 9 different filter -> for making different perspectives when youu do matmul over the input
      #kernel_size=3,strides=2

  tile_filter_op = tf.constant(tile_filter,dtype=tf.float32)# cast to tf and note that this is constant
 
  output = tf.nn.depthwise_conv2d(input,tile_filter_op,strides=[1,stride,stride,1],padding="VALID") #None,28,28,8*17 * 3,3,8*17,3*3 =>(None, 13, 13, 1224)

  output = tf.reshape(output,shape=[batch_size,output.shape[1],output.shape[2],input_channel,kernel_size*kernel_size]) #None,13,13,8*17,3*3

  output = tf.transpose(output,perm=[0,1,2,4,3]) #None,13,13,3*3,8*17

  model = Model(inputs=input,outputs=output,name=name+"_kernel_tile")
  return model

In [7]:
def variable_with_regularizer(shape,element_size,initializer,regularizer,name):
  w = initializer # use this to make some variable
  w = tf.map_fn(fn=lambda t: regularizer(t), elems=tf.reshape(w,[element_size,1]),name=name+"_regularized_l2_w") # make them outside so I can regularize all 
 
  w = tf.reshape(w,shape,name=name+"_mat_trasform_w_reshape") # reshape into w_shape
  return w

In [8]:
def mat_transform(pose_shape,caps_num_c,regularizer,name,tag=False):
  #input_shape == None*13*13,3*3*8,16 first conv_caps_layer as example
  #caps_num_c == votes_shape # output_dim = vote_dim == ABCD
  #caps_num_i = 3,3*3*B B=8
  new_batch = pose_shape[0]
  input_shape = pose_shape[1:]
  input = Input(batch_size=new_batch,shape=input_shape,name=name+"_mat_transform_input") # batch_size*13*13,3*3*8,16
  caps_num_i = pose_shape[1] #3*3*B
  ##
  
  output = tf.reshape(input,[new_batch,caps_num_i,1,4,4],name=name+"_mat_trasform_output_reshape") #batch_size*13*13,3*3*8,1,4,4
  output = tf.tile(output,[1,1,caps_num_c,1,1],name=name+"_mat_trasform_output_tile") #batch_size*13*13,3*3*8,A/B/C/D,4,4
  ##
  w_shape = [1,caps_num_i,caps_num_c,4,4] # weighting shape == 1,3*3*A/B/C/D,A/B/C/D,4,4
  element_size = 1*caps_num_i*caps_num_c*4*4
  ##
  initializer = tf.random.truncated_normal(shape=w_shape)
  regularizer = tf.keras.regularizers.L2(5e-4)
  w = variable_with_regularizer(w_shape,element_size,initializer,regularizer,name)
  w = tf.tile(w,[new_batch,1,1,1,1],name=name+"_mat_trasform_w_tile") #new_batch_size,3*3*A/B/C/D,A/B/C/D,4,4
  ##
  votes = tf.reshape(output@w,[new_batch,caps_num_i,caps_num_c,16],name=name+"_votes") # #new_batch_size,3*3*A/B/C/D,A/B/C/D,16 as input_shape
  model = Model(inputs=input,outputs=votes,name=name+"_mat_transform")
  return model

In [9]:

def em_routing(votes,activation,new_batch_size,caps_num_i,caps_num_c,n_channels,regularizer,name,tag=False):
  # vote shape == #new_batch_size,3*3*A/B/C/D,A/B/C/D,16  #new_batch_size,3*3*8,8,16
  # acitvation_shape == #new_batch_size*13*13,3*3*8,1
  # new_batch_size = activation_shape[0]
  # caps_num_i = activation_shape[1] #
  # n_channels = votes_shape[-1] #16

 

  print("caps_num_c : {}, n_channels : {}".format(caps_num_c,n_channels))
  # set up initializer
 
  sigma_square = []
  miu = []
  activation_out = []
  ###
  beta_v_shape = [caps_num_c,n_channels] #a/b/c/d,16 ==> 8,16
  beta_v_element_size = caps_num_c*n_channels
  beta_v = variable_with_regularizer(beta_v_shape,beta_v_element_size, tf.constant(0,tf.float32,shape=beta_v_shape),regularizer,name+"_beta_v")
  ###

  beta_a = variable_with_regularizer([caps_num_c],caps_num_c,tf.constant(0,tf.float32,shape=caps_num_c),regularizer,name+"_beta_a") #caps_num_c ==> 8

  
  votes_in = votes
  activation_in = activation

  for iters in range(iter_routing):
    if iters == 0:
      r = tf.constant(np.ones([new_batch_size,caps_num_i,caps_num_c])/caps_num_c,tf.float32,name=name+"_r") # r == batch_size*13*13,3*3*8,8
    else:
      log_p_c_h = -tf.math.log(tf.sqrt(sigma_square)) - (tf.square(votes_in-miu)/ (2*sigma_square)) ##batch_size*13*13,3*3*8,8,16
      log_p_c_h -= tf.reduce_max(log_p_c_h,axis=[2,3],keepdims=True) - tf.math.log(10.0) ##batch_size*13*13,3*3*8,1,1 => ##batch_size*13*13,3*3*8,8,16(boardCast)
      p_c = tf.exp(tf.reduce_sum(log_p_c_h,axis=3)) ##batch_size*13*13,3*3*8,8
      ap = p_c * tf.reshape(activation_out,shape=[new_batch_size,1,caps_num_c]) ## batch_size*13*13,3*3*8,8 * batch_size*13*13,1,8=> batch_size*13*13,3*3*8,8 * batch_size*13*13,,3*3*8,8(board_cast) => batch_size*13*13,,3*3*8,8
      r = ap/(tf.reduce_sum(ap,axis=2,keepdims=True)+epsilon) #..todo batch_size*13*13,3*3*8,8,16

    r = r*activation_in #batch_size*13*13,3*3*8,8,1
    r = r/(tf.reduce_sum(r,axis=2,keepdims=True)+epsilon) #batch_size*13*13,3*3*8,1 
    r_sum = tf.reduce_sum(r,axis=1,keepdims=True) #batch_size*13*13,1,8 
    r1 = tf.reshape(r/(r_sum+epsilon),shape=[new_batch_size,caps_num_i,caps_num_c,1]) # #batch_size*13*13,3*3*8,8,1
    miu = tf.reduce_sum(votes_in*r1,axis=1,keepdims=True) #new_batch_size*13*13,3*3*8,8,16 * batch_size*13*13,3*3*8,8,16(boardcasted) => batch_size*13*13,1,8,16=>batch_size*13*13,3*3*8,8,16
    sigma_square = tf.reduce_sum(tf.square(votes_in-miu)*r1,axis=1,keepdims=True)+epsilon #batch_size*13*13,1,8,16 => #batch_size*13*13,3*3*8,8,16(boardcasted)
    
    if iters == iter_routing-1:
      r_sum = tf.reshape(r_sum,[new_batch_size,caps_num_c,1]) #batch_size*13*13,3*3*8,8,1 => batch_size*13*13,8,1 #since keepdim => can squeeze like this
      cost_h = (beta_v+tf.math.log(tf.sqrt(tf.reshape(sigma_square,shape=[new_batch_size,caps_num_c,n_channels])))) * r_sum  #  batch_size*13*13,8,16(sigma_square) * batch_size*13*13,8,1 => batch_size*13*13,8,16
      activation_out = tf.nn.softmax(ac_lambda0*(beta_a-tf.reduce_sum(cost_h,axis=2))) # ..todo new_batch_size,8,16 => new_batch_size,8,1(redcue_sum)
    else:
      activation_out = tf.nn.softmax(r_sum) ##batch_size*13*13,3*3*8,8,16
  

  return miu,activation_out

In [10]:
def calEm(input_pose,input_activation,pose_shape,activation_shape,votes_output_dim,weights_regularizer,name):
  new_batch_size,caps_num_i = input_pose.shape[:2]
  # caps_num_c == votes_output_dim
  # pose shape  #None*13*13,3*3*8,17
  # calculation on voting
  ## model 1
  mat_transform_model = mat_transform(pose_shape,votes_output_dim,weights_regularizer,name=name)
  
  votes = mat_transform_model(input_pose)
  ## model 2
  # vote shape ==  # #new_batch_size*13*13,3*3*A/B/C/D,A/B/C/D,16 
  # acitvation_shape == #new_batch_size*13*13,3*3*8,1
  caps_num_c = votes_output_dim
  n_channels = votes.shape[-1]
  miu,activation = em_routing(votes,input_activation,new_batch_size,caps_num_i,caps_num_c,n_channels,weights_regularizer,name=name)

  return miu,activation
 
  
  

In [11]:
# import gc
# gc.collect()
# input_pose = np.ones([10*13*13,3*3*8,16])
# input_activation = np.ones([10*13*13,3*3*8,1])
# v,e = calEm(input_pose,input_activation,[10*13*13,3*3*8,16],[10*13*13,3*3*8,1],8,tf.keras.regularizers.l2(5e-4),"try") 

In [12]:
def conv_caps(input,input_shape,kernel_size,stride,output_dim,weights_regularizer,name):
  # output_dim = vote_dim == ABCD
  # define kernel_tile_model
  kernel_tile_model = build_kernel_tile(input_shape[1:],kernel_size,stride,name)
 
  # output from kernel_tile_model
  output = kernel_tile_model(input) #None,13,13,3*3,8*17
  
  data_size = input_shape[1]
  data_size = int(np.floor(data_size-kernel_size+1)/stride)
 
  new_batch = batch_size * data_size * data_size #None*13*13
  # make it easier to cut the tensors into 2 part

  output = tf.reshape(output,shape=[new_batch,-1,17]) #None*13*13,3*3*8,17
  activation = tf.reshape(output[:,:,16],[new_batch,-1,1]) #None*13*13,3*3*8,1
  # output[:,:,:16] is pose of last layer
  # activation is last layer activation

  #calEm(input_pose,input_activation,[10*13*13,3*3*8,16],[10*13*13,3*3*8,1],8,tf.keras.regularizers.l2(5e-4),"try") 
  #mat_transform_model,em_routing_model = calEM(output.shape[:,:,:16],activation.shape,output_dim,weights_regularizer,name) #output shape is #None*13*13,3*3*8,16
  pose = output[:,:,:16]
  
  miu,activation = calEm(pose,activation,pose.shape,activation.shape,output_dim,weights_regularizer,name)

  #pose
  pose = tf.reshape(miu,[batch_size,data_size,data_size,output_dim,16])
  activation = tf.reshape(activation,[batch_size,data_size,data_size,output_dim,1])

  
  return pose,activation

In [13]:
def conv_caps_1(output_from_primary_caps,weights_regularizer):
  input = layers.Input(batch_size=batch_size,shape=output_from_primary_caps.shape[1:])
  pose,activation = conv_caps(input,output_from_primary_caps.shape,3,2,C,weights_regularizer,"conv_caps_1_conv")
  data_size = pose.shape[1]
  output = tf.reshape(tf.concat([pose,activation],axis=4),[batch_size,data_size,data_size,C*17])
  model = Model(inputs=[input],outputs=[output],name="conv_caps_1")
  return model

In [14]:
def conv_caps_2(output_from_primary_caps,weights_regularizer):
  input = layers.Input(batch_size=batch_size,shape=output_from_primary_caps.shape[1:])
  pose,activation = conv_caps(input,output_from_primary_caps.shape,3,1,D,weights_regularizer,"conv_caps_1_conv")
  data_size = pose.shape[1]
  #class caps
  pose = tf.reshape(pose,[batch_size*3*3,D,16])
  activation = tf.reshape(activation,[batch_size*3*3,D,-1])
  miu,activation = calEm(pose,activation,pose.shape,activation.shape,num_classes,weights_regularizer,"conv_caps_2")
  activation = tf.reshape(activation,[batch_size,data_size,data_size,num_classes])
  miu = tf.reshape(miu,[batch_size,data_size,data_size,-1])
  output = tf.nn.avg_pool(activation,ksize=[1,data_size,data_size,1],strides=[1,1,1,1],padding="VALID")
  output = tf.reshape(output,[batch_size,num_classes])
  pose = tf.nn.avg_pool(miu,ksize=[1,data_size,data_size,1],strides=[1,1,1,1],padding="VALID")
  pose_out = tf.reshape(pose,shape=[batch_size,num_classes,16])
  model = Model(inputs=[input],outputs=[output,pose_out],name="conv_caps_2")
  return model

In [15]:
def spread_loss(output,pose_out,x,y,m):

  data_size = x.shape[1]
  y = tf.one_hot(y,num_classes,dtype=tf.float32)

  reshape_output = tf.reshape(output,[tf.shape(y)[0],1,num_classes])

  y = tf.expand_dims(y,axis=2)

  at = reshape_output@y

  loss = tf.square(tf.maximum(0.,m-(at-reshape_output)))
  loss = loss@(1.-y)
  loss = tf.reduce_mean(loss)

  pose_out = tf.reshape(tf.multiply(pose_out,y),shape=[batch_size,-1])

  model = models.Sequential(
      [
       layers.Dense(512,trainable=True,kernel_regularizer=tf.keras.regularizers.l2(5e-4)),
       layers.Dense(1024,trainable=True,kernel_regularizer=tf.keras.regularizers.l2(5e-4)),
       layers.Dense(data_size*data_size,trainable=True,kernel_regularizer=tf.keras.regularizers.l2(5e-4),activation="sigmoid")
      ]
  )
  pose_out = model(pose_out)
  x = tf.reshape(x,shape=[tf.shape(y)[0],-1])
  reconstruction_loss = tf.reduce_mean(tf.square(pose_out-x)) #6800 - 25??
  loss_all = tf.add_n([loss]+[0.0005*data_size*data_size*reconstruction_loss])
  return loss_all,reconstruction_loss,pose_out


In [16]:
def test_accuracy(logits,labels):
  
  logits_index = tf.cast((tf.argmax(logits,axis=1)),tf.int32)
  
  #logits_index = tf.reshape(logits_index,labels.shape[0])
  #labels = tf.reshape(labels,labels.shape[0])

  correct_preditions = tf.equal(tf.cast(labels,tf.int32),logits_index)
  accuracy =tf.reduce_sum(tf.cast(correct_preditions,tf.float32)) / tf.cast(tf.shape(labels)[0],tf.float32)
  return accuracy,logits_index

In [17]:
def create_model():
  #input 1 
  input = Input(shape=[28,28,1],batch_size=batch_size)
  batch_x_squash = tf.divide(input,255.)
  batch_x = layers.BatchNormalization(center=False,trainable=True)(batch_x_squash) 
  #input 2
  batch_labels = Input(shape=[],batch_size=batch_size)

  #model start
  model_1, data_size_1 = scope_1(shape=[28,28,1])
  model_2, data_size_2 = scope_2(*data_size_1)
  model_3 = conv_caps_1(model_2.output,weights_regularizer)
  model_4 = conv_caps_2(model_3.output,weights_regularizer)
  
  x = model_1(batch_x)
  x = model_2(x)
  x = model_3(x)

  output,pose_out = model_4(x)

 
  
  batch_labels_casted = tf.cast(batch_labels,tf.int32)
  
  #loss,_,_ = spread_loss(output,pose_out,batch_x_squash,batch_labels_casted,m)


  model = Model(inputs=[input,batch_labels],outputs=[output,pose_out,batch_x_squash],name="EM_Routing_capsnet")
  #accuracy,logits_index = test_accuracy(output,batch_labels_casted)

  return model
 

In [18]:
def loss(model,input,batch_labels):
  output,pose_out,batch_x_squash = model(input)
  loss,_,_ = spread_loss(output,pose_out,batch_x_squash,batch_labels,m)
  return loss

In [19]:
def grad(model, inputs, batch_labels):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs,batch_labels)
  return loss_value, tape.gradient(loss_value,model.trainable_variables)


In [20]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.77.202.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.77.202.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [21]:
m_min = 0.2
m_max = 0.9
m = m_min
decay_steps = 1000
decay_rate = 0.8
##HYPERPERAMETER
weight_reg = False
epsilon = 1e-9
iter_routing = 2
A,B,C,D = 32,8,16,1 #each layers output channel dim
batch_size = 5000
num_classes = 10
ac_lambda0 = 0.01
weights_regularizer = tf.keras.regularizers.l2(5e-4)

In [ ]:
model = create_model()

In [ ]:
model.summary()

In [ ]:
(X_train,y_train),(X_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
def get_df(X,y):
  y = np.int32(y)
  X = np.float32(X)
  X_expanded = np.expand_dims(X,3)
  X_tf = tf.data.Dataset.from_tensor_slices(X_expanded)
  y_tf = tf.data.Dataset.from_tensor_slices(y)
  total_tf = tf.data.Dataset.zip((X_tf,y_tf))
  df = total_tf.repeat().batch(batch_size)
  return df

In [ ]:
train_dataset  = get_df(X_train,y_train)
test_dataset  = get_df(X_test,y_test)

In [ ]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(1e-3,decay_steps=decay_steps,end_learning_rate=1e-5)
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [ ]:
## training start
train_loss_results = []
train_accuracy_results = []
num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    loss_value,grads = grad(model,x,y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    epoch_loss_avg(loss_value)
    output,_,_ = model(x)
    epoch_accuracy(y,output)

  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 5 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result()))
